### Challenge 11:  An ECB/CBC detection oracle

<div class="alert alert-block alert-info">
    
Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:

```encryption_oracle(your-input)```
    
```=> [MEANINGLESS JIBBER JABBER]```
    
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.

</div>
    
[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
from numpy.random import randint
import cryptopals as cp

In [2]:
def encryption_oracle(data):
    """
    Implements Set 2, Challenge 11 - Encryption Oracle
    Emulates an accessible function that will encrypt user-provided data under an unknown (random) key
    For the purposes of this exercise, it randomly selects between ECB and CBC mode and returns the "truth"
    Data about what mode was used so we can see if we correctly detect it...
    """
    key = bytes(list(randint(0, 256, 16)))
    mode = randint(0, 2)
    prepend_data = bytes(list(randint(0, 256, randint(5, 11))))
    append_data = bytes(list(randint(0, 256, randint(5, 11))))

    data = prepend_data + data + append_data

    if mode == 0:

        true_mode = 'CBC'
        ciphertext = cp.AESEncrypt(data, key, 'CBC')

    elif mode == 1:

        true_mode = 'ECB'
        ciphertext = cp.AESEncrypt(data, key, 'ECB')

    ECB_flag = cp.detect_AES_ECB(ciphertext)

    if ECB_flag:
        detected_mode = 'ECB'
    else:
        detected_mode = 'CBC'

    return(ciphertext, true_mode, detected_mode)

In [4]:
# Play with the length of the plaintext we supply to see when the detection oracle starts 
# being successful...

N_bytes = 44
N_loops = 1000

for N_bytes in range(35, 45):

    plaintext = b'c'*N_bytes
    success_count = 0

    for ii in range(N_loops):

        [ciphertext, true_mode] = cp.encryption_oracle(plaintext)
        
        # Test for ECB...
        ECB_flag = cp.detect_AES_ECB(ciphertext)
        if ECB_flag:
            detected_mode = 'ECB'
        else:
            detected_mode = 'CBC'
        
        # Does our test result match the true mode?
        if (detected_mode == true_mode):
            success_count += 1

    print(f"N_bytes={N_bytes}:  {success_count} Successful detections out of {N_loops}")

N_bytes=35:  507 Successful detections out of 1000
N_bytes=36:  491 Successful detections out of 1000
N_bytes=37:  504 Successful detections out of 1000
N_bytes=38:  586 Successful detections out of 1000
N_bytes=39:  666 Successful detections out of 1000
N_bytes=40:  750 Successful detections out of 1000
N_bytes=41:  828 Successful detections out of 1000
N_bytes=42:  933 Successful detections out of 1000
N_bytes=43:  1000 Successful detections out of 1000
N_bytes=44:  1000 Successful detections out of 1000


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)